In [76]:
from pulp import *
import pandas as pd
import networkx as nx
import pandas as pd
#Heavily influenced by, need to make sure we understand the code so we can use it and not plagerize as a final product
#https://www.kaggle.com/code/itoeiji/solving-tsp-and-vrp-by-mip-using-pulp

In [57]:
distEnum = {
    "SFO": { "SFO": 0, "LAX": 500, "SJC": 75, "OAK": 50},
    "LAX": { "SFO": 500, "LAX": 0, "SJC": 280, "OAK": 800},
    "SJC": { "SFO": 75, "LAX": 280, "SJC": 0, "OAK": 30},
    "OAK": { "SFO": 50, "LAX": 800, "SJC": 30, "OAK": 0}
}


distDict = pd.DataFrame( 
    [
    [0, 500, 75, 50],
    [500, 0, 280, 800],
    [75, 280, 0, 30],
    [50, 800, 30, 0]
    ])

distDict2 = pd.DataFrame( 
    [
    [0, 9999, 469, 857, 375],  #MSY
    [9999, 0, 549, 1188, 464], #IND
    [469, 549, 0, 639, 380],   #ATL
    [857, 1188, 639, 0, 9999], #FLL
    [375, 464, 380, 9999, 0]   #MEM
    ])

In [77]:
usRoutes = pd.read_csv('../data/usRoutesClean.csv')

In [284]:
def findRoutes (dataSet, startAirport, endAirport, cutoff):
    # Create the network so that we can get our additional routes that could be used to get from our start to finish
    G = nx.from_pandas_edgelist(dataSet, 'SRCIATA', 'DESTINIATA', create_using=nx.DiGraph())

    cutoffValue = 0     # Used to track what depth the while loop is at
    noRoute = True      # Used to track if a route was found and the loop can be stopped early

    while (cutoffValue <= cutoff) & (noRoute == True):  # Loop goes through and finds the shallowest depth that we can work with
        cutoffValue = cutoffValue + 1
        paths = nx.all_simple_paths(G, startAirport, endAirport, cutoff=cutoffValue)

        for path in paths:
            noRoute = False

    if cutoffValue > cutoff:
        return(pd.DataFrame())

    paths = nx.all_simple_paths(G, startAirport, endAirport, cutoff=cutoffValue)

    shortestTotalDistance = 0   # Used to track what the shortest total distance is for later comparison of ranking 

    for path in paths: 
        nodeLocation = 0
        totalDistance = 0
        potentialRoute = pd.DataFrame({"SRC": [], "DESTIN": [], "METERS": []})

        for index in path:
            # Calculate total distance of the route
            currentIATA = index
            if nodeLocation != 0:
                distance = (dataSet.loc[(dataSet['SRCIATA'] == pastIATA) & (dataSet['DESTINIATA'] == currentIATA), 'meters'].values)[0]
                totalDistance = totalDistance + distance
                indexEntry = pd.DataFrame({"SRC": [pastIATA], "DESTIN": [currentIATA], "METERS": [distance]})
                potentialRoute = pd.concat([potentialRoute, indexEntry])

            nodeLocation = nodeLocation + 1
            pastIATA = index
        
        if shortestTotalDistance == 0:                  # Takes the first value found a promotes it to the shortest
            shortestTotalDistance = totalDistance
            shortestRoute = potentialRoute.copy()
        else:
            if shortestTotalDistance > totalDistance:   # Compares the shortest with the current to see if it needs to replace
                shortestTotalDistance = totalDistance
                shortestRoute = potentialRoute.copy()

    return(shortestRoute)

In [294]:
def sampleRandomAirports(n):
    sampleAirports = pd.Series(usRoutes['SRCIATA'].sample(n))
    i = 0
    while (i != len(sampleAirports)):
        j = 0
        while (j <= len(sampleAirports) - 1):
            route = findRoutes(usRoutes, sampleAirports.iloc[i], sampleAirports.iloc[j], 2)
            if (route.empty):
                pass
            elif (route['DESTIN'].iloc[0] != sampleAirports.iloc[j]):
                newDestin = pd.Series(route['DESTIN'].iloc[0])
                if(pd.concat([sampleAirports, newDestin]).is_unique):  
                    sampleAirports = pd.concat([sampleAirports, newDestin]) 
            else:
                DISTANCEList.append((route['METERS'].iloc[0]/1609.344))
            j = j + 1
        i = i + 1
    return sampleAirports

In [313]:
def createDistanceMatrix(sampleAirports):
    DistanceMatrix = []
    i = 0
    while (i != len(sampleAirports)):
        DISTANCEList = []
        j = 0
        while (j <= len(sampleAirports) - 1):
            route = findRoutes(usRoutes, sampleAirports.iloc[i], sampleAirports.iloc[j], 2)

            if (sampleAirports.iloc[i] == sampleAirports.iloc[j]):
                DISTANCEList.append(0)
            elif ((route.empty) or (route['DESTIN'].iloc[0] != sampleAirports.iloc[j])):
                DISTANCEList.append(9999)
            elif (route['DESTIN'].iloc[0] != sampleAirports.iloc[j]):
                DISTANCEList.append(9999)
            else:
                DISTANCEList.append((route['METERS'].iloc[0]/1609.344))
            
            j = j + 1
        DistanceMatrix.append(DISTANCEList)
        i = i + 1

    DistanceMatrix = pd.DataFrame(DistanceMatrix, columns = sampleAirports, index = sampleAirports)
    DistanceMatrix = DistanceMatrix.fillna(0)
    DistanceMatrix = DistanceMatrix.round(0)
    DistanceMatrix = DistanceMatrix.astype(int)
    return DistanceMatrix

In [319]:
distMatrix = createDistanceMatrix(sampleRandomAirports(3))
print(distMatrix)

      SLC   GUA   BWI   DFW   ATL
SLC     0  9999  2091  1226  1889
GUA  9999     0  9999  1822  2416
BWI  2089  9999     0  1381   680
DFW  1225  1821  1381     0   808
ATL  1890  2419   680   807     0


In [320]:
problem = LpProblem("LP_Opt_AC", LpMinimize)

x = LpVariable.dicts('x', ((i, j) for i in range(len(distMatrix)) for j in range(len(distMatrix))), lowBound=0, upBound=1, cat='Binary')

problem += pulp.lpSum(distMatrix[i][j] * x[i, j] for i in range(len(distMatrix)) for j in range(len(distMatrix)))

u = pulp.LpVariable.dicts('u', (i for i in range(len(distMatrix))), lowBound=1, upBound=9999, cat='Integer')

for i in range(len(distMatrix)):
    problem += x[i, i] == 0

for i in range(len(distMatrix)):
    problem += pulp.lpSum(x[i, j] for j in range(len(distMatrix))) == 1
    problem += pulp.lpSum(x[j, i] for j in range(len(distMatrix))) == 1

for i in range(len(distMatrix)):
    for j in range(len(distMatrix)):
        if i != j and (i != 0 and j != 0):
            problem += u[i] - u[j] <= len(distMatrix) * (1 - x[i, j]) - 1

KeyError: 0

In [321]:
status = problem.solve()
status, LpStatus[status], value(problem.objective)

AttributeError: 'NoneType' object has no attribute 'value'